# The following is the project on Segmenting and Clustering of Neighborhoods in Toronto

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from bs4 import BeautifulSoup
import lxml
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  54.41 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.87 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  40.91 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  49.38 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |###################

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url).text

soup = BeautifulSoup(page, 'lxml')

table = soup.find('table', class_='wikitable sortable')
table.prettify

<bound method Tag.prettify of <table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a

In [4]:
tableheader = table.find_all('tr')

tablehead = []
for t1 in tableheader:
    t2 = t1.find_all('th')
    row1 = [t1.text.strip() for t1 in t2 if t1.text.strip()]
    
    if row1:
        tablehead.append(row1)

df1 = pd.DataFrame(tablehead)
df1

,0,1,2
0,Postcode,Borough,Neighbourhood


In [5]:
tablerows = table.find_all('tr')

tablebody = []
for t1 in tablerows:
    t2 = t1.find_all('td')
    row2 = [t1.text.strip() for t1 in t2 if t1.text.strip()]
    if row2:
        tablebody.append(row2)

df2 = pd.DataFrame(tablebody) 
df2.head()

,0,1,2
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
df3 = pd.concat([df1, df2])

df3.columns = df3.iloc[0]

df3.head()

,Postcode,Borough,Neighbourhood
0,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village


In [7]:
df3 = df3[1:]

df3.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [8]:
df3 = df3.drop(df3[df3["Borough"] == "Not assigned"].index)
df3

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [9]:
df3.replace("Not assigned", np.nan, inplace=True)
df3.ffill(axis =1)

df = df3

df['Postcode'] = df['Postcode'].astype(str)
df['Borough'] = df['Borough'].astype(str)
df['Neighbourhood'] = df['Neighbourhood'].astype(str)

df.set_index(['Postcode', 'Borough'], inplace=True)
df = df.groupby(level=['Postcode', 'Borough'], sort=False).agg( ','.join)

df = df.reset_index()

df.shape

(103, 3)

In [10]:
geocoder = pd.read_csv('https://cocl.us/Geospatial_data')
df = df.join(geocoder)
df

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",M1E,43.763573,-79.188711
3,M6A,North York,"Lawrence Heights,Lawrence Manor",M1G,43.770992,-79.216917
4,M7A,Queen's Park,nan,M1H,43.773136,-79.239476
5,M9A,Etobicoke,Islington Avenue,M1J,43.744734,-79.239476
6,M1B,Scarborough,"Rouge,Malvern",M1K,43.727929,-79.262029
7,M3B,North York,Don Mills North,M1L,43.711112,-79.284577
8,M4B,East York,"Woodbine Gardens,Parkview Hill",M1M,43.716316,-79.239476
9,M5B,Downtown Toronto,"Ryerson,Garden District",M1N,43.692657,-79.264848


In [11]:
from geopy.geocoders import Nominatim 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

In [12]:
address = 'TORONTO'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The lati & long coordinate of Toronto are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

The lati & long coordinate of Toronto are 43.653963, -79.387207.


In [14]:
downtowntoronto_data = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)

address = 'Downtown TORONTO'

geolocator = Nominatim(user_agent="dtt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The lati & long coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

map_downtowntoronto = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, label in zip(downtowntoronto_data['Latitude'], downtowntoronto_data['Longitude'], downtowntoronto_data['Neighbourhood']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_downtowntoronto)

map_downtowntoronto

The lati & long coordinate of Downtown Toronto are 43.6541737, -79.3808116451341.


In [15]:
CLIENT_ID = '3XW3TOWZPHZQKTEJABYEEEADSGTDEQI1COQRUGHX1WM4340R' 
CLIENT_SECRET = 'JG2OGEETAOAETUEAV2DILFQVJT4Y0BAVMBQHUNUFI5NEWK33' 
VERSION = '20190211' 
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3XW3TOWZPHZQKTEJABYEEEADSGTDEQI1COQRUGHX1WM4340R
CLIENT_SECRET:JG2OGEETAOAETUEAV2DILFQVJT4Y0BAVMBQHUNUFI5NEWK33


In [16]:
downtowntoronto_data.loc[1, 'Neighbourhood']

'Ryerson,Garden District'

In [18]:
neighbourhood_latitude = downtowntoronto_data.loc[1, 'Latitude'] 
neighbourhood_longitude = downtowntoronto_data.loc[1, 'Longitude'] 

neighborhood_name = downtowntoronto_data.loc[1, 'Neighbourhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Ryerson,Garden District are 43.692657000000004, -79.2648481.


In [19]:
LIMIT = 100
radius = 500 


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=3XW3TOWZPHZQKTEJABYEEEADSGTDEQI1COQRUGHX1WM4340R&client_secret=JG2OGEETAOAETUEAV2DILFQVJT4Y0BAVMBQHUNUFI5NEWK33&v=20190211&ll=43.692657000000004,-79.2648481&radius=500&limit=100'

In [20]:
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5c725a6cf594df76ba438272'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-56d51743498e60da346470e2-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16d941735',
         'name': 'Café',
         'pluralName': 'Cafés',
         'primary': True,
         'shortName': 'Café'}],
       'id': '56d51743498e60da346470e2',
       'location': {'address': '1666 Kingston Road',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Birchcliff Avenue',
        'distance': 113,
        'formattedAddress': ['1666 Kingston Road (Birchcliff Avenue)',
         'Toronto ON M1N 1S5',
         'Canada'],
        'labeledLatLngs': [{'label

In [21]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = result['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Birchcliff,Café,43.691666,-79.264532
1,Birchmount Community Centre,General Entertainment,43.695175,-79.262161
2,Scarborough Gardens,Skating Rink,43.694647,-79.262230
3,Birchmount Stadium,College Stadium,43.695323,-79.261293


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
dtt_venues = getNearbyVenues(names=downtowntoronto_data['Neighbourhood'],
                                   latitudes=downtowntoronto_data['Latitude'],
                                   longitudes=downtowntoronto_data['Longitude']
                                  )

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city
Church and Wellesley


In [27]:
print(dtt_venues.shape)
dtt_venues.head()

(153, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
1,"Harbourfront,Regent Park",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Harbourfront,Regent Park",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
3,"Harbourfront,Regent Park",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
4,"Harbourfront,Regent Park",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location


In [28]:
dtt_onehot = pd.get_dummies(dtt_venues[['Venue Category']], prefix="", prefix_sep="")

dtt_onehot['Neighbourhood'] = dtt_venues['Neighbourhood'] 

fixed_columns = [dtt_onehot.columns[-1]] + list(dtt_onehot.columns[:-1])
dtt_onehot = dtt_onehot[fixed_columns]

dtt_grouped = dtt_onehot.groupby('Neighbourhood').mean().reset_index()
dtt_grouped

,Neighbourhood,Airport,Auto Workshop,Bakery,Bar,Baseball Field,Beer Store,Bookstore,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Chinese Restaurant,Coffee Shop,College Stadium,Comic Shop,Convenience Store,Cosmetics Shop,Curling Ice,Dessert Shop,Diner,Discount Store,Electronics Store,Empanada Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,French Restaurant,Garden,Garden Center,Gastropub,General Entertainment,Gourmet Shop,Grocery Store,Gym,Home Service,Ice Cream Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Other Repair Shop,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Recording Studio,Rental Car Location,Restaurant,Sandwich Place,School,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Social Club,Spa,Steakhouse,Supplement Shop,Sushi Restaurant,Tea Room,Vegetarian / Vegan Restaurant,Video Store,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.25,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.00,0.055556,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.000000,0.055556,0.000000,0.000000,0.000000,0.00,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.055556,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.055556,0.000000,0.000000,0.055556,0.000000,0.000000,0.055556,0.000000,0.055556,0.000000,0.000000,0.055556,0.000000,0.055556,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556
2,"Cabbagetown,St. James Town",0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.250000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Chinatown,Gr

In [29]:
num_top_venues = 5

for hood in dtt_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = dtt_grouped[dtt_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
               venue  freq
0            Airport  0.25
1               Park  0.25
2  Other Repair Shop  0.25
3           Bus Stop  0.25
4                Pub  0.00


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                  venue  freq
0           Yoga Studio  0.06
1               Butcher  0.06
2  Fast Food Restaurant  0.06
3      Recording Studio  0.06
4            Restaurant  0.06


----Cabbagetown,St. James Town----
                 venue  freq
0          Pizza Place   0.5
1  Empanada Restaurant   0.5
2              Airport   0.0
3         Home Service   0.0
4             Pharmacy   0.0


----Central Bay Street----
         venue  freq
0  Pizza Place  0.25
1     Pharmacy  0.25
2  Coffee Shop  0.25
3      Butcher  0.25
4      Airport  0.00


----Chinatown,Grange Park,Kensington Market----
                venue  freq
0         Coffee Shop  0.11
1         Pizza Place  0.08
2               

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = dtt_grouped['Neighbourhood']

for ind in np.arange(dtt_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dtt_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

kclusters = 5

dtt_grouped_clustering = dtt_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dtt_grouped_clustering)

neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Airport,Park,Other Repair Shop,Bus Stop,Farmers Market,Diner,Discount Store,Electronics Store,Empanada Restaurant,Falafel Restaurant
1,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Yoga Studio,Skate Park,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Park,Garden Center,Pizza Place,Recording Studio
2,"Cabbagetown,St. James Town",Pizza Place,Empanada Restaurant,Gourmet Shop,Curling Ice,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market
3,Central Bay Street,Pizza Place,Pharmacy,Coffee Shop,Butcher,Curling Ice,Dessert Shop,Diner,Discount Store,Electronics Store,Empanada Restaurant
4,"Chinatown,Grange Park,Kensington Market",Coffee Shop,Café,Pizza Place,Sushi Restaurant,Italian Restaurant,French Restaurant,Gastropub,Falafel Restaurant,Gourmet Shop,Electronics Store


In [33]:
dtt_merged = downtowntoronto_data


dtt_merged = dtt_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

dtt_merged.head() 

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",M1E,43.763573,-79.188711,1.0,Pizza Place,Rental Car Location,Mexican Restaurant,Medical Center,Spa,Breakfast Spot,Electronics Store,Falafel Restaurant,Dessert Shop,Diner
1,M5B,Downtown Toronto,"Ryerson,Garden District",M1N,43.692657,-79.264848,1.0,General Entertainment,College Stadium,Café,Skating Rink,Farmers Market,Diner,Discount Store,Electronics Store,Empanada Restaurant,Falafel Restaurant
2,M5C,Downtown Toronto,St. James Town,M1W,43.799525,-79.318389,1.0,Grocery Store,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Sandwich Place,Japanese Restaurant,Electronics Store,Pharmacy,Pizza Place,Breakfast Spot
3,M5E,Downtown Toronto,Berczy Park,M2L,43.757490,-79.374714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M5G,Downtown Toronto,Central Bay Street,M2R,43.782736,-79.442259,1.0,Pizza Place,Pharmacy,Coffee Shop,Butcher,Curling Ice,Dessert Shop,Diner,Discount Store,Electronics Store,Empanada Restaurant


In [34]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dtt_merged['Latitude'], dtt_merged['Longitude'], dtt_merged['Neighbourhood'], dtt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow,
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [35]:
#Cluster 1
dtt_merged.loc[dtt_merged['Cluster Labels'] == 0, dtt_merged.columns[[1] + list(range(5, dtt_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,-79.498509,0.0,Home Service,Baseball Field,Yoga Studio,Fish & Chips Shop,Diner,Discount Store,Electronics Store,Empanada Restaurant,Falafel Restaurant,Farmers Market


In [36]:
#Cluster 2
dtt_merged.loc[dtt_merged['Cluster Labels'] == 1, dtt_merged.columns[[1] + list(range(5, dtt_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,-79.188711,1.0,Pizza Place,Rental Car Location,Mexican Restaurant,Medical Center,Spa,Breakfast Spot,Electronics Store,Falafel Restaurant,Dessert Shop,Diner
1,Downtown Toronto,-79.264848,1.0,General Entertainment,College Stadium,Café,Skating Rink,Farmers Market,Diner,Discount Store,Electronics Store,Empanada Restaurant,Falafel Restaurant
2,Downtown Toronto,-79.318389,1.0,Grocery Store,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Sandwich Place,Japanese Restaurant,Electronics Store,Pharmacy,Pizza Place,Breakfast Spot
4,Downtown Toronto,-79.442259,1.0,Pizza Place,Pharmacy,Coffee Shop,Butcher,Curling Ice,Dessert Shop,Diner,Discount Store,Electronics Store,Empanada Restaurant
5,Downtown Toronto,-79.329656,1.0,Fast Food Restaurant,Park,Food & Drink Shop,Farmers Market,Dessert Shop,Diner,Discount Store,Electronics Store,Empanada Restaurant,Falafel Restaurant
6,Downtown Toronto,-79.464763,1.0,Airport,Park,Other Repair Shop,Bus Stop,Farmers Market,Diner,Discount Store,Electronics Store,Empanada Restaurant,Falafel Restaurant
7,Downtown Toronto,-79.318389,1.0,Skating Rink,Beer Store,Pharmacy,Curling Ice,Bus Stop,Park,Spa,Cosmetics Shop,Video Store,Food
8,Downtown Toronto,-79.315572,1.0,Sandwich Place,Pet Store,Burrito Place,Fish & Chips Shop,Fast Food Restaurant,Gym,Ice Cream Shop,Italian Restaurant,Liquor Store,Movie Theater
10,Downtown Toronto,-79.476013,1.0,Convenience Store,Skating Rink,Bar,Discount Store,Sandwich Place,Dessert Shop,Diner,Electronics Store,Empanada Restaurant,Falafel Restaurant
11,Downtown Toronto,-79.484450,1.0,Coffee Shop,Café,Pizza Place,Sushi Restaurant,Italian Restaurant,French Restaurant,Gastropub,Falafel Restaurant,Gourmet Shop,Electronics Store


In [37]:
#Cluster 3
dtt_merged.loc[dtt_merged['Cluster Labels'] == 2, dtt_merged.columns[[1] + list(range(5, dtt_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Downtown Toronto,-79.532242,2.0,Baseball Field,Yoga Studio,Fish & Chips Shop,Diner,Discount Store,Electronics Store,Empanada Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant
